# Notebook 03: Rerank - "The Precision Logic"

## Innocenti Risk Management Enablement Kit

---

### The Problem

Vector search finds semantically similar documents, but it doesn't understand **nuance**:

- "Can police use facial recognition?" matches both:
  - Article 5 (General prohibition)
  - Article 5.1.d (Specific exceptions for law enforcement)

A lawyer needs the **specific exception**, not just "similar" documents.

### The Solution: Listwise Reranking

**Traditional Reranking (Pointwise):**
- Scores each document independently against the query
- Doesn't consider relationships between documents

**Listwise Reranking (Jina Reranker v3):**
- Documents "attend" to each other during scoring
- Understands context: "This document is more specific than that one"
- The "Lawyer's Brain" effect

**What we'll do:**
1. Retrieve top 50 results ("naive" retrieval)
2. Apply Jina Reranker v3 via Elasticsearch's `retrievers` API
3. Compare before/after rankings

---

## 1. Setup & Dependencies

In [ ]:
!pip install -q elasticsearch python-dotenv pandas

from utils.colab_setup import setup_environment
IN_COLAB = setup_environment(packages="elasticsearch python-dotenv pandas")

In [ ]:
from pathlib import Path
from elasticsearch import Elasticsearch, BadRequestError
import pandas as pd

# Import our credential helper
# (Path is already set correctly in previous cell for both Colab and local)
from utils.credentials import setup_notebook, get_index_name, get_inference_id, get_elasticsearch_client

# Better pandas display
pd.set_option('display.max_colwidth', 60)
pd.set_option('display.width', 200)

print("✓ Libraries loaded successfully!")

In [ ]:
# Setup credentials and display configuration
creds = setup_notebook(require_elastic=True, require_jina=False)

# Get unique names for this user
INDEX_NAME = get_index_name("search-eu-ai-act")
EMBEDDING_ID = get_inference_id("embeddings")
RERANKER_ID = get_inference_id("reranker")

In [ ]:
# Connect to Elasticsearch (works with both Cloud and Serverless)
es = get_elasticsearch_client(creds)

# Verify connection and index
info = es.info()
print(f"✓ Connected to Elasticsearch {info['version']['number']}")

if es.indices.exists(index=INDEX_NAME):
    count = es.count(index=INDEX_NAME)["count"]
    print(f"✓ Index {INDEX_NAME} has {count} documents")
else:
    raise ValueError(f"Index {INDEX_NAME} not found. Run Notebook 02 first.")

## 2. Create Reranker Inference Endpoint

We need to set up the Jina Reranker v3 model as an inference endpoint.

**Jina Reranker v3 highlights:**
- **Listwise** attention mechanism
- Documents attend to each other, not just the query
- Better at distinguishing "general" vs "specific" matches

In [ ]:
from utils.inference import create_reranker_inference

In [ ]:
# Create the reranker inference endpoint
create_reranker_inference(es, RERANKER_ID)

## 3. Define Our Test Query

We'll use a nuanced legal query that has both general and specific answers in the EU AI Act.

In [ ]:
# A nuanced query where context matters
QUERY = "Can law enforcement use facial recognition in public spaces?"

# What we expect:
# - Article 5: General prohibition on biometric systems
# - Article 5.1.d: EXCEPTIONS for law enforcement (the nuanced answer)

print(f"Test Query: \"{QUERY}\"")
print("\nExpected behavior:")
print("  - Vector search: Returns general prohibition articles")
print("  - Reranked: Should surface the law enforcement EXCEPTIONS")

## 4. Naive Retrieval (Vector Search Only)

First, let's see what pure semantic search returns - no reranking.

In [ ]:
def search_naive(es_client, index: str, query: str, size: int = 10) -> list:
    """
    Perform naive semantic search without reranking.
    
    Args:
        es_client: Elasticsearch client
        index: Index name
        query: Search query
        size: Number of results
    
    Returns:
        List of hit dictionaries
    """
    results = es_client.search(
        index=index,
        query={
            "semantic": {
                "field": "text",
                "query": query
            }
        },
        size=size,
        source=["article_number", "title", "text"],
    )
    return results['hits']['hits']

In [ ]:
import time

# Get naive results
_t0 = time.time()
naive_results = search_naive(es, INDEX_NAME, QUERY, size=10)
naive_elapsed_ms = (time.time() - _t0) * 1000

print(f"--- Naive Semantic Search Results ({naive_elapsed_ms:.0f} ms) ---")
print(f"Query: \"{QUERY}\"\n")

naive_data = []
for i, hit in enumerate(naive_results, 1):
    naive_data.append({
        "Rank": i,
        "Article": hit['_source']['article_number'],
        "Title": hit['_source']['title'][:50] + "..." if len(hit['_source']['title']) > 50 else hit['_source']['title'],
        "Score": f"{hit['_score']:.4f}"
    })

df_naive = pd.DataFrame(naive_data)
print(df_naive.to_string(index=False))

## 5. Reranked Retrieval (Using `retrievers` API)

Now we use Elasticsearch's native `retrievers` parameter with `text_similarity_reranker`.

This is the **recommended** way to do reranking in Elasticsearch:
- Single request (no separate API call)
- Native integration
- Optimized for performance

**How it works:**
1. Inner retriever: `standard` query gets initial candidates
2. Outer retriever: `text_similarity_reranker` re-scores using the reranker model

In [ ]:
def search_with_reranker(es_client, index: str, query: str, 
                         reranker_id: str, initial_size: int = 50, 
                         final_size: int = 10) -> list:
    """
    Perform semantic search with native reranking via retrievers API.
    
    Args:
        es_client: Elasticsearch client
        index: Index name
        query: Search query
        reranker_id: Inference endpoint ID for reranker
        initial_size: Number of candidates for reranking
        final_size: Final number of results to return
    
    Returns:
        List of hit dictionaries
    """
    results = es_client.search(
        index=index,
        retriever={
            "text_similarity_reranker": {
                "retriever": {
                    "standard": {
                        "query": {
                            "semantic": {
                                "field": "text",
                                "query": query
                            }
                        }
                    }
                },
                "field": "text",
                "inference_id": reranker_id,
                "inference_text": query,
                "rank_window_size": initial_size,
            }
        },
        size=final_size,
        source=["article_number", "title", "text"],
    )
    return results['hits']['hits']

In [ ]:
# Get reranked results
print("Running reranked search...")
print(f"  Initial candidates: 50")
print(f"  Reranker: {RERANKER_ID}")

_t0 = time.time()
reranked_results = search_with_reranker(
    es, INDEX_NAME, QUERY, 
    reranker_id=RERANKER_ID,
    initial_size=50,
    final_size=10
)
reranked_elapsed_ms = (time.time() - _t0) * 1000

print(f"\n--- Reranked Results ({reranked_elapsed_ms:.0f} ms) ---")
print(f"Query: \"{QUERY}\"\n")

reranked_data = []
for i, hit in enumerate(reranked_results, 1):
    reranked_data.append({
        "Rank": i,
        "Article": hit['_source']['article_number'],
        "Title": hit['_source']['title'][:50] + "..." if len(hit['_source']['title']) > 50 else hit['_source']['title'],
        "Score": f"{hit['_score']:.4f}"
    })

df_reranked = pd.DataFrame(reranked_data)
print(df_reranked.to_string(index=False))

## 6. Side-by-Side Comparison

Let's visualize how the rankings changed.

In [ ]:
from utils.comparison import build_comparison

In [ ]:
# Display comparison
print("=" * 70)
print("  RANKING COMPARISON: Naive vs Reranked")
print("=" * 70)
print(f"Query: \"{QUERY}\"")
print()

df_comparison = build_comparison(naive_results, reranked_results)
print(df_comparison.to_string(index=False))

print()
print("Legend: ↑ = moved up, ↓ = moved down, = = same, NEW = wasn't in top 10")

In [ ]:
assert len(df_comparison) == 10, f"Expected 10 comparison rows, got {len(df_comparison)}"
assert "Movement" in df_comparison.columns
print("✓ Comparison validation passed")

## 7. Deep Dive: Why Reranking Matters

Let's look at the actual text to understand why the reranker made different choices.

In [ ]:
# Show text snippets from top 3 reranked results
print("=" * 70)
print("  TOP 3 RERANKED RESULTS - Why They Ranked Higher")
print("=" * 70)

for i, hit in enumerate(reranked_results[:3], 1):
    print(f"\n#{i}: Article {hit['_source']['article_number']} - {hit['_source']['title']}")
    print("-" * 60)
    
    # Get text content - handle semantic_text structure
    text = hit['_source']['text']
    if isinstance(text, dict):
        text = text.get('text', str(text))
    
    # Show first 400 chars
    preview = text[:400] if len(text) > 400 else text
    print(preview + "..." if len(text) > 400 else preview)

## 8. Quantitative Analysis

Let's measure the impact of reranking on our results.

In [ ]:
# Calculate reranking impact metrics
naive_articles = [h['_source']['article_number'] for h in naive_results]
reranked_articles = [h['_source']['article_number'] for h in reranked_results]

# How many articles changed position?
position_changes = sum(1 for i, art in enumerate(reranked_articles) 
                       if i < len(naive_articles) and naive_articles[i] != art)

# How many new articles appeared in top 10?
new_in_top10 = len(set(reranked_articles) - set(naive_articles))

# Overlap between naive and reranked top 5
top5_overlap = len(set(naive_articles[:5]) & set(reranked_articles[:5]))

print("=" * 50)
print("  RERANKING IMPACT METRICS")
print("=" * 50)
print(f"  Position changes in top 10:  {position_changes}/10")
print(f"  New articles in top 10:      {new_in_top10}")
print(f"  Top-5 overlap:               {top5_overlap}/5")
print("=" * 50)

---

## Summary: The Full Chain Advantage

You've now seen the complete **"Full Chain"** search pipeline:

| Step | Tool | What It Does |
|------|------|-------------|
| **1. Ingest** | Jina Reader (ReaderLM) | PDF → Clean Markdown |
| **2. Embed** | Jina Embeddings v5 (EIS) | Text → Semantic Vectors |
| **3. Rerank** | Jina Reranker v3 (EIS) | Precision via Listwise Attention |

### Key Takeaways

| Concept | What We Learned |
|---------|----------------|
| **Listwise Reranking** | Documents attend to each other, not just the query |
| **retrievers API** | Native Elasticsearch integration for reranking |
| **Precision vs Recall** | Vector search has recall, reranking adds precision |
| **Legal Nuance** | Reranking surfaces exceptions and specifics |

### The "Innocenti" Advantage

For a legal compliance firm:
- **Before:** "Yes, facial recognition is banned" (general answer)
- **After:** "Facial recognition is banned EXCEPT for law enforcement in specific circumstances" (precise answer)

That precision can be the difference between compliance and a violation.

---

## Next Steps

**Layer B: The Demo UI**

The Next.js application will:
1. Connect to this same index
2. Let users search the EU AI Act
3. Have a "Deep Analysis" button that triggers reranking
4. Visually show the ranking changes

---